In [153]:
!pip install pyviz

  Using cached numba-0.52.0-cp37-cp37m-macosx_10_14_x86_64.whl (2.2 MB)
  Using cached llvmlite-0.35.0-cp37-cp37m-macosx_10_9_x86_64.whl (18.5 MB)
ERROR: fastparquet 0.5.0 has requirement pandas>=1.1.0, but you'll have pandas 0.24.2 which is incompatible.
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.33.0+1.g022ab0f
ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [154]:
import os
import pandas as pd
import numpy as np
import talib as ta
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime as dt
import hvplot.pandas
import ccxt
%matplotlib inline

In [155]:
tickers = ["BBY", "UAL", "DISCK", "FANG", "NEE"]

In [156]:
load_dotenv()
# Import API Variable Keys
## Stocks
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")


In [157]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Format current date as ISO format
#today = pd.Timestamp(now, tz="America/New_York").isoformat()

# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "15Min"
limit=50
end_date = pd.Timestamp.now(tz="America/New_York").isoformat()


df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    limit,
    end = end_date
).df

df_portfolio.index = df_portfolio.index.date
df_portfolio = df_portfolio.dropna()
df_portfolio.tail()

BBY                                       DISCK          \
               open     high      low    close   volume    open    high   
2021-03-05  102.065  102.800  102.040  102.780   8404.0  52.800  52.920   
2021-03-05  102.860  103.100  102.670  102.850   7196.0  52.965  53.030   
2021-03-05  102.870  102.960  102.585  102.730   7911.0  53.015  53.030   
2021-03-05  102.750  103.145  102.690  103.085  15171.0  52.965  53.095   
2021-03-05  103.045  103.190  102.840  102.975  14987.0  52.990  53.250   

                                     ...     NEE                       \
               low   close   volume  ...    open   high    low  close   
2021-03-05  52.730  52.910   2807.0  ...  69.775  70.17  69.72  70.07   
2021-03-05  52.900  53.010  16588.0  ...  70.055  70.24  70.00  70.00   
2021-03-05  52.915  52.985  21277.0  ...  69.990  70.35  69.99  70.34   
2021-03-05  52.895  53.005   8071.0  ...  70.400  70.75  70.28  70.75   
2021-03-05  52.990  53.005  18127.0  ...  70.715  70.99  70.63  70.66   

                       UAL                                   
             volume   open    high     low   close   volume  
2021-03-05  16159.0  49.85  50.110  49.840  49.955   4523.0  
2021-03-05  13125.0  49.92  50.180  49.870  50.170   9684.0  
2021-03-05  14159.0  50.20  50.515  50.180  50.365   9497.0  
2021-03-05  12129.0  50.37  50.455  50.145  50.295  29405.0  
2021-03-05  22719.0  50.30  50.650  50.215  50.650  51292.0  

[5 rows x 25 columns]

In [158]:
df_bby = pd.DataFrame()
df_bby = df_portfolio['BBY']
df_bby.tail()

,open,high,low,close,volume
2021-03-05,102.065,102.800,102.040,102.780,8404.0
2021-03-05,102.860,103.100,102.670,102.850,7196.0
2021-03-05,102.870,102.960,102.585,102.730,7911.0
2021-03-05,102.750,103.145,102.690,103.085,15171.0
2021-03-05,103.045,103.190,102.840,102.975,14987.0


In [159]:
def applyTA (input_df):
    """
        Add columns of technical analysis in the dataframe
        Parameter:
            input_df - dataframe with date index and columns of OHLCV data
        Returns:
            df_ta - dataframe with date index and technical analysis signals
    """
    
    df = input_df.copy()
    
    # Construct EMA Singals
    df['8D_EMA'] = ta.EMA(df['close'], timeperiod = 8)
    df['21D_EMA'] = ta.EMA(df['close'], timeperiod = 21)
    df['50D_EMA'] = ta.EMA(df['close'], timeperiod = 50)
    df['8_21_x_long'] = np.where(df['8D_EMA'] > df['21D_EMA'], 1.0, 0.0)
    df['8_21_x_short'] = np.where(df['8D_EMA'] < df['21D_EMA'], -1.0, 0.0)
    df['8_21_x_signal'] = df['8_21_x_long'] + df['8_21_x_short'] 
    df['21_50_x_long'] = np.where(df['21D_EMA'] > df['50D_EMA'], 1.0, 0.0)
    df['21_50_x_short'] = np.where(df['21D_EMA'] < df['50D_EMA'], -1.0, 0.0)
    df['21_50_x_signal'] = df['21_50_x_long'] + df['21_50_x_short']
    df['above_8D'] = np.where(df['close'] > df['8D_EMA'], 1.0, 0.0)
    df['above_21D'] = np.where(df['close'] > df['21D_EMA'], 1.0, 0.0)
    df['above_50D'] = np.where(df['close'] > df['50D_EMA'], 1.0, 0.0)
    
    # Bollinger Bands
    df['bb_upper_band'], df['bb_middle_band'], df['bb_lower_band'] = ta.BBANDS(df['close'], timeperiod =20)
    df['bb_long'] = np.where(df['close'] < df['bb_lower_band'], 1.0, 0.0)
    df['bb_short'] = np.where(df['close'] > df['bb_upper_band'], -1.0, 0.0)
    df['bb_signal'] = df['bb_long'] + df['bb_short']
    
    # Calculate Stochastic Oscillators trading signal
    df['slowk'], df['slowd'] = ta.STOCH(df['high'], df['low'], df['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)   
    df['slowk_x_long'] = np.where(df['slowk'] > df['slowd'], 1.0, 0.0)
    df['slowk_x_short'] = np.where(df['slowk'] < df['slowd'], -1.0, 0.0)
    df['slowk_x_signal'] = df['slowk_x_long'] + df['slowk_x_short']
    
     # Calculate ADX
    df['adx'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['adx_strong'] = np.where(df['adx'] > 50, 1.0, 0.0)
    df['adx_weak'] = np.where(df['adx'] < 20, 1.0, 0.0)

    # Calculate OBV signal
    df['obv'] = ta.OBV(df['close'], df['volume'])
    df['obv_change']=df['obv'].pct_change()
    df['obv_increase'] = np.where(df['obv_change'] > 0, 1.0, 0.0)

    # Calculate Chaikin A/D Oscillator signal
    df['ADOSC'] = ta.ADOSC(df['high'], df['low'], df['close'], df['volume'], fastperiod=3, slowperiod=10)    
    df['adosc_positive'] = np.where(df['ADOSC'] > 0, 1.0, 0.0)

    # Drop non-TA signals columns and NAs
    df_ta = df[{'8_21_x_signal', '21_50_x_signal', 'above_8D', 'above_21D', 'above_50D', 'bb_signal', 'slowk_x_signal', 'adx_strong', 'adx_weak', 'obv_increase', 'adosc_positive'}]
    df_ta = df_ta.dropna()
    
    return df_ta

In [160]:
applyTA(df_bby).tail()

,slowk_x_signal,adx_weak,above_21D,above_50D,adx_strong,21_50_x_signal,adosc_positive,obv_increase,bb_signal,above_8D,8_21_x_signal
2021-03-05,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0


In [161]:
df_TA_bby = applyTA(df_bby)
df_TA_bby.tail()

,slowk_x_signal,adx_weak,above_21D,above_50D,adx_strong,21_50_x_signal,adosc_positive,obv_increase,bb_signal,above_8D,8_21_x_signal
2021-03-05,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0


In [162]:
df_TA_bby.reset_index(inplace=True)

In [163]:
for row in range(len(df_TA_bby)):
    df_TA_bby.loc[row, 'total_score'] = df_TA_bby.loc[row, '8_21_x_signal'] + df_TA_bby.loc[row, '21_50_x_signal'] + df_TA_bby.loc[row,'above_8D'] + df_TA_bby.loc[row, 'above_21D'] + df_TA_bby.loc[row,'above_50D'] + df_TA_bby.loc[row,'bb_signal'] + df_TA_bby.loc[row,'slowk_x_signal'] + df_TA_bby.loc[row,'adx_strong'] + df_TA_bby.loc[row,'adx_weak'] + df_TA_bby.loc[row,'obv_increase'] + df_TA_bby.loc[row,'adosc_positive']
df_TA_bby.tail()

,index,slowk_x_signal,adx_weak,above_21D,above_50D,adx_strong,21_50_x_signal,adosc_positive,obv_increase,bb_signal,above_8D,8_21_x_signal,total_score
44,2021-03-05,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,7.0
45,2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,5.0
46,2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,4.0
47,2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,5.0
48,2021-03-05,-1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,4.0


In [164]:
df_TA_bby = df_TA_bby.drop(columns=['above_21D', 'above_50D', 'obv_increase', 'bb_signal', 'adx_weak', 'adosc_positive', 'adx_strong', 'above_8D', '8_21_x_signal', '21_50_x_signal', 'slowk_x_signal'])
df_TA_bby.tail()

,index,total_score
44,2021-03-05,7.0
45,2021-03-05,5.0
46,2021-03-05,4.0
47,2021-03-05,5.0
48,2021-03-05,4.0


In [165]:
df_bby = df_bby.drop(columns=['open', 'high', 'low', 'volume'])

In [166]:
df_bby.tail()

,close
2021-03-05,102.780
2021-03-05,102.850
2021-03-05,102.730
2021-03-05,103.085
2021-03-05,102.975


In [167]:
df_TA_bby.index = df_TA_bby['index']

In [168]:
df_TA_bby.tail()

,index,total_score
index,,
2021-03-05,2021-03-05,7.0
2021-03-05,2021-03-05,5.0
2021-03-05,2021-03-05,4.0
2021-03-05,2021-03-05,5.0
2021-03-05,2021-03-05,4.0


In [169]:
df_TA_bby = df_TA_bby.drop(columns=['index'])
df_TA_bby.index.name = 'date'
df_TA_bby.tail()

,total_score
date,
2021-03-05,7.0
2021-03-05,5.0
2021-03-05,4.0
2021-03-05,5.0
2021-03-05,4.0


In [170]:
df_bby.index.name = 'date'
df_bby.tail()

,close
date,
2021-03-05,102.780
2021-03-05,102.850
2021-03-05,102.730
2021-03-05,103.085
2021-03-05,102.975


In [171]:
df_concat = pd.concat([df_bby , df_TA_bby], axis='columns', join='outer')
df_concat.tail()

,close,total_score
date,,
2021-03-05,102.780,7.0
2021-03-05,102.850,5.0
2021-03-05,102.730,4.0
2021-03-05,103.085,5.0
2021-03-05,102.975,4.0


In [172]:
df_concat = df_concat.reset_index()
df_concat.tail()

,date,close,total_score
44,2021-03-05,102.780,7.0
45,2021-03-05,102.850,5.0
46,2021-03-05,102.730,4.0
47,2021-03-05,103.085,5.0
48,2021-03-05,102.975,4.0


In [173]:
# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
#signals_df["Signal"][short_window:] = np.where(
#    signals_df["SMA50"][short_window:] < signals_df["SMA100"][short_window:], 1.0, 0.0
#)

#df_concat['signal'] = 0.0
for rows in range(len(df_concat["date"])):
    if df_concat.loc[rows, 'total_score'] >=3:
        df_concat.loc[rows, "signal"] = 1
    elif df_concat.loc[rows, 'total_score'] <= -3:
        df_concat.loc[rows, "signal"] = -1
    else:
        df_concat.loc[rows, "signal"] = 0

# Calculate the points in time at which a position should be taken, 1 or -1
df_concat["Entry/Exit"] = df_concat["signal"].diff()

# Print the DataFrame
df_concat.dropna(inplace=True)
df_concat

,date,close,total_score,signal,Entry/Exit
1,2021-03-04,98.025,1.0,0.0,0.0
2,2021-03-04,98.120,1.0,0.0,0.0
3,2021-03-04,97.640,0.0,0.0,0.0
4,2021-03-04,97.850,1.0,0.0,0.0
5,2021-03-04,98.660,1.0,0.0,0.0
6,2021-03-04,0.000,0.0,0.0,0.0
7,2021-03-04,98.880,2.0,0.0,0.0
8,2021-03-04,0.000,-1.0,0.0,0.0
9,2021-03-04,98.450,4.0,1.0,1.0
10,2021-03-04,97.800,3.0,1.0,0.0


In [174]:
#df_concat.set_index(df_concat["date"], inplace=True)

In [175]:
# Visualize exit position relative to close price
entry = df_concat[df_concat["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color="green", legend=False, ylabel="Price in $", width=1000, height=400
)

# Visualize entry position relative to close price
exit = df_concat[df_concat["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color="red", legend=False, ylabel="Price in $", width=1000, height=400
)

# Visualize close price for the investment
security_close = df_concat["close"].hvplot(
    line_color="lightgray", ylabel="Price in $", width=1000, height=400
)

# Overlay plots
entry_exit_plot = security_close * entry * exit
entry_exit_plot

:Overlay
   .Curve.Close      :Curve   [index]   (close)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

In [178]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = -500

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
df_concat["Position"] = share_size * df_concat["signal"]

# Find the points in time where a 500 share position is bought or sold
df_concat["Entry/Exit Position"] = df_concat["Position"].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
df_concat["Portfolio Holdings"] = (
    df_concat["close"] * df_concat["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
df_concat["Portfolio Cash"] = (
    initial_capital - (df_concat["close"] * df_concat["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
df_concat["Portfolio Total"] = (
    df_concat["Portfolio Cash"] + df_concat["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
df_concat["Portfolio Daily Returns"] = df_concat["Portfolio Total"].pct_change()

# Calculate the cumulative returns
df_concat["Portfolio Cumulative Returns"] = (
    1 + df_concat["Portfolio Daily Returns"]
).cumprod() - 1

In [180]:
# Print the DataFrame
df_concat

,date,close,total_score,signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
1,2021-03-04,98.025,1.0,0.0,0.0,-0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-04,98.120,1.0,0.0,0.0,-0.0,0.0,0.0,100000.0,100000.0,NaN,NaN
3,2021-03-04,97.640,0.0,0.0,0.0,-0.0,0.0,0.0,100000.0,100000.0,0.000000,0.000000
4,2021-03-04,97.850,1.0,0.0,0.0,-0.0,0.0,0.0,100000.0,100000.0,0.000000,0.000000
5,2021-03-04,98.660,1.0,0.0,0.0,-0.0,0.0,0.0,100000.0,100000.0,0.000000,0.000000
6,2021-03-04,0.000,0.0,0.0,0.0,-0.0,0.0,0.0,100000.0,100000.0,0.000000,0.000000
7,2021-03-04,98.880,2.0,0.0,0.0,-0.0,0.0,0.0,100000.0,100000.0,0.000000,0.000000
8,2021-03-04,0.000,-1.0,0.0,0.0,-0.0,0.0,0.0,100000.0,100000.0,0.000000,0.000000
9,2021-03-04,98.450,4.0,1.0,1.0,-500.0,-500.0,-49225.0,149225.0,100000.0,0.000000,0.000000
10,2021-03-04,97.800,3.0,1.0,0.0,-500.0,0.0,-48900.0,149225.0,100325.0,0.003250,0.003250


In [179]:
df_concat# Visualize exit position relative to total portfolio value
entry = df_concat[df_concat["Entry/Exit"] == -1.0]["Portfolio Total"].hvplot.scatter(
    color="red", legend=False, ylabel="Total Portfolio Value", width=1000, height=400
)

# Visualize entry position relative to total portfolio value
exit = df_concat[df_concat["Entry/Exit"] == 1.0]["Portfolio Total"].hvplot.scatter(
    color="green", legend=False, ylabel="Total Portfolio Value", width=1000, height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = df_concat[["Portfolio Total"]].hvplot(
    line_color="lightgray", ylabel="Total Portfolio Value", width=1000, height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [index]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [index]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [index]   (Portfolio Total)